In [2]:
import xarray as xr
import numpy  as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt

In [165]:
# 读取数据
# year = "2006"
# 注释掉的是原始数据，此后的是进行了mask之后的数据

dir_vr = "/raid52/yycheng/MPAS/VR_postprocess/VR_merge/diag_mean_mon/" + "monmean_2003-2015_VR_diag.nc"
ds_vr = xr.open_dataset(dir_vr)

dir_rcm = "/raid52/yycheng/MPAS/RCM_postprocess/RCM_merge/diag_mean_mon/" + "monmean_2003-2015_RCM_diag.nc"
ds_rcm = xr.open_dataset(dir_rcm)

dir_erai="/raid52/yycheng/MPAS/REFERENCE/ERA5I_NC/ERA5I_NC_monthy"
ds_era_temp = xr.open_dataset(dir_erai + "/sel_air.mon.mean.nc")
ds_era_dewtemp = xr.open_dataset(dir_erai + "/sel_air.mon.mean.nc")
ds_era_rhum = xr.open_dataset(dir_erai + "/sel_rhum.mon.mean.nc")
ds_era_shum = xr.open_dataset(dir_erai + "/sel_shum.mon.mean.nc")
ds_era_u    = xr.open_dataset(dir_erai + "/sel_uwnd.mon.mean.nc")
ds_era_v    = xr.open_dataset(dir_erai + "/sel_vwnd.mon.mean.nc")
ds_era_hgt  = xr.open_dataset(dir_erai + "/sel_hgt.mon.mean.nc")

# era5i water flux
era_vinwvf = xr.open_dataset(dir_erai + "/sel_vinwvf.mon.mean.nc")['vinwvf']
era_viewvf = xr.open_dataset(dir_erai + "/sel_viewvf.mon.mean.nc")['viewvf']



In [28]:
vr_relhum_850 = ds_vr['relhum_850hPa']

In [32]:
Rv = 461.5 # J * kg-1 * K-1
vr_e_850 = vr_relhum_850 * 850 * 100 # 水汽压
rho_vapor_vr_850 = vr_e_850 / Rv / ds_vr['temperature_850hPa'] # 水汽密度 kg * m-3

In [38]:
vr_u_flux = rho_vapor_vr_850 * ds_vr['uzonal_850hPa']
vr_v_flux = rho_vapor_vr_850 * ds_vr['umeridional_850hPa']

计算一边ERAI的water flux，确认计算过程没有错误，可以用ERA5I的资料直接进行对比

计算Niu文章中的 water vapor transport (unit: m*s-1*g*kg-1 at 850 hPa)

In [205]:
t_count = ds_era_temp['air'].sel(level=850) - 273.15               # 计算需要使用摄氏度
era_e_850_sa = 6.112 * np.exp( 17.67 * t_count / (t_count + 243.5) )  # 850hPa饱和水汽压，单位hPa
era_e_850 = era_e_850_sa * ds_era_rhum['rhum'].sel(level=850) / 100.   # 850hPa水汽压，单位hPa
# era_e_850 = era_e_850_sa   # 850hPa水汽压，单位hPa
# 水汽密度尝试下计算，失败
era_q_850 = 622 * (era_e_850 / (850-0.378 * era_e_850))          # 850hPa比湿，单位g*kg

In [206]:
# check shum
np.max((ds_era_shum['shum'].sel(level=850).values * 1000 - era_q_850) / era_q_850)

<xarray.DataArray ()>
array(0.19640988)
Coordinates:
    level    float64 850.0